In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
import seaborn as sn

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
df_heart = pd.read_csv('/kaggle/input/heart-failure-clinical-data/heart_failure_clinical_records_dataset.csv')

In [ ]:
df_heart.head(10)

In [ ]:
df_heart.info()

In [ ]:
df_heart.describe()

In [ ]:
plt.figure(figsize=(10, 8))
sn.heatmap(df_heart.corr(),vmin=-1, vmax=1, annot=True)
plt.show()

In [ ]:
#salvando os labels
labels = np.array(df_heart['DEATH_EVENT'])

#salvando a ordem das features
features_list = list(df_heart.columns)

In [ ]:
#dropando a variável target
df_heart.drop('DEATH_EVENT', axis=1, inplace=True)

#verificando se foi realmente excluída
df_heart.columns

In [ ]:
#convertendo o df para array
data = np.array(df_heart)

In [ ]:
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.feature_selection import SelectFromModel
from sklearn.svm import LinearSVC
#importando bibliotecas para sintonia de hiperparâmetros
from sklearn.model_selection import StratifiedKFold, RandomizedSearchCV, GridSearchCV
from scipy.stats import uniform, randint
from sklearn.metrics import f1_score, make_scorer

In [ ]:
#definindo o tipo de validação cruzada e número de folds manualmente
cv_strat = StratifiedKFold(n_splits=2, shuffle=True, random_state=1)

for train_index, test_index in cv_strat.split(data, labels):
    x_train, x_test = data[train_index], data[test_index]
    y_train, y_test = labels[train_index], labels[test_index]

# **Random Forest**

In [ ]:
from sklearn.ensemble import RandomForestClassifier

In [ ]:
rf_model = Pipeline(steps=[('random', RandomForestClassifier())])

In [ ]:
rf_model.fit(x_train, y_train)

In [ ]:
rf_model.score(x_test, y_test)

In [ ]:
rf_model.get_params().keys()

In [ ]:
# Tunando hiperparâmetros com k-fold StratifiedKFold e pipelines
parameters = {
                'random__n_estimators': (range(100, 201)),
                'random__bootstrap': [True, False]
}
rf_model_test = Pipeline(steps=[('random', RandomForestClassifier(random_state=42))])
grid = GridSearchCV(rf_model_test, param_grid=parameters, cv=cv_strat, n_jobs=-1)
grid.fit(data, labels)

In [ ]:
#vamos olhar para os melhores resultados encontrados pelo Grid Search
print('Melhor resultado f1:', grid.best_score_)
print('\n\nMelhor configuração de hiperparâmetros:', grid.best_params_)

print( '\n\nConfigurações de todos os hiperparâmetros do melhor estimado encontrado pelo GridSearch: \n', grid.best_estimator_)